In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

chat = ChatOpenAI(
  model_name="gpt-3.5-turbo",
  openai_api_key=openai_api_key,
  temperature=0.1,
  streaming=True,
  callbacks=[StreamingStdOutCallbackHandler()]
)

examples = [
  {
    "movie": "The Matrix",
    "answer": """
    About 'The Matrix'
    Produced by:  The Wachowskis
    Starring:     Keanu Reeves
                  Laurence Fishburne
                  Carrie-Anne Moss
                  Hugo Weaving
                  Joe Pantoliano
    Budget:       $63 million
    Box office:   $467.6 million
    """,
  },
  {
    "movie": "Cast Away",
    "answer": """
    About 'Cast Away'
    Produced by:  Robert Zemeckis
    Starring:     Tom Hanks
                  Helen Hunt
                  Nick Searcy
    Budget:       $90 million
    Box office:   $429.6 million
    """,
  },
  {
    "movie": "WALL-E",
    "answer": """
    About 'WALL-E'
    Produced by:  Andrew Stanton
    Starring:     Ben Burtt
                  Elissa Knight
                  Jeff Garlin
                  Fred Willard
                  John Ratzenberger
                  Kathy Najimy
                  Sigourney Weaver
    Budget:       $180 million
    Box office:   $532.5 million
      """,
  },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Can you explain me about {movie}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie critic, explain the movie."),
        example_prompt,
        ("human", "Can you explain me about {movie}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"movie": "The Batman"})

About 'The Batman'
Directed by:  Matt Reeves
Starring:     Robert Pattinson
              Zoe Kravitz
              Paul Dano
              Jeffrey Wright
              Colin Farrell
Budget:       $180 million (estimated)
Box office:   $1.1 billion (projected)
Release date: March 4, 2022

'The Batman' is a dark and gritty reboot of the iconic superhero franchise, focusing on a younger and more brooding version of the Caped Crusader. Robert Pattinson stars as Bruce Wayne/Batman, who is still in the early stages of his crime-fighting career and is grappling with his own inner demons. The film follows Batman as he investigates a series of gruesome murders in Gotham City, leading him to confront the enigmatic serial killer known as The Riddler, played by Paul Dano. Zoe Kravitz portrays Selina Kyle/Catwoman, a skilled thief who becomes entangled in Batman's quest for justice. With a stellar cast, intense action sequences, and a dark and atmospheric tone, 'The Batman' offers a fresh and comp

AIMessageChunk(content="About 'The Batman'\nDirected by:  Matt Reeves\nStarring:     Robert Pattinson\n              Zoe Kravitz\n              Paul Dano\n              Jeffrey Wright\n              Colin Farrell\nBudget:       $180 million (estimated)\nBox office:   $1.1 billion (projected)\nRelease date: March 4, 2022\n\n'The Batman' is a dark and gritty reboot of the iconic superhero franchise, focusing on a younger and more brooding version of the Caped Crusader. Robert Pattinson stars as Bruce Wayne/Batman, who is still in the early stages of his crime-fighting career and is grappling with his own inner demons. The film follows Batman as he investigates a series of gruesome murders in Gotham City, leading him to confront the enigmatic serial killer known as The Riddler, played by Paul Dano. Zoe Kravitz portrays Selina Kyle/Catwoman, a skilled thief who becomes entangled in Batman's quest for justice. With a stellar cast, intense action sequences, and a dark and atmospheric tone, '